In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from datetime import datetime
from itertools import product

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=2) # kavi1234 paper-trade
# ib.connect('127.0.0.1', 4003, clientId=3) # ***kavi1234 IBG  LIVE ***
# ib.connect('127.0.0.1', 7498, clientId=4) # ***kavi1234 TWS  LIVE ***

<IB connected to 127.0.0.1:4004 clientId=2>

In [2]:
%%time
# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

# **** Make list of qualified options stocks from the web (23 secs) *****

options_url1 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=1"
options_url2 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=2"
nse = pd.concat([pd.read_html(options_url1, header=0)[2], 
                         pd.read_html(options_url2, header=0)[2]], 
                        sort=True).drop_duplicates(subset='Symbol').reset_index(drop=True)

nse = nse.iloc[:, [1, 2, 3]]
nse.columns = ['iSymbol', 'Desc', 'Symbol']

# Note multipliers are not correct for NSE!. So get them from 5paisa!!!

paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol').reset_index(drop=True)  # It's the second table in the url
paisa_small = paisa[['Symbol', 'Mlot']].sort_values('Symbol') # small df only with Mlot

# Check for symbols missing in nse
# pd.merge(paisa_small, nse, on='Symbol', how='left', indicator=True).query("_merge == 'left_only'" )

nse_options = pd.merge(paisa_small, nse, on="Symbol", how='left')
nse_options["Exchange"] = 'NSE'

nse_options.loc[nse_options.iSymbol.isnull(), "iSymbol"] = nse_options['Symbol'].str.slice(0,9)

# List of nse indexes (extracted from error!)
idx_list = ["BANKNIFTY", "NIFTY50", "NSEFTSE", "USDINR", "INDIAVIX", 
            "NIFTYCPSE", "NIFTYINFR", "NIFTYIT", "NIFTYMID5", "NIFTYPSE"]
nse_options = nse_options.loc[~nse_options.iSymbol.isin(idx_list), :]

# Get the Stocks qualified
stocks = [Stock(symbol=x, exchange='NSE') for x in nse_options.iSymbol]
qual_stock = ib.qualifyContracts(*stocks)

# Put the contracts to a Contract column
nse_options = nse_options.assign(Contract = pd.Series(qual_stock).values)

Wall time: 25 s


In [45]:
%%time
# **** Get the close price of Underlying (20 seconds) ****

def get_price(contract):
    '''Gets the price of contract
    Args:
       (contract)    = object: qualified contract object
       (exchange)    = string: ['NSE']
    
    Returns:
        price        = float
    '''
    
    m_data = ib.reqMktData(contract, snapshot=True)
    
    # In case reqMktData doesn't load the prices
    if m_data.time == None:
        m_data = ib.reqMktData(contract)
        ib.sleep(0.05)
        m_data = ib.reqMktData(contract)
        ib.cancelMktData(contract)

    return m_data.close

list_of_prices = [get_price(c) for c in qual_stock]

nse_options["Und_Price"] = list_of_prices

Wall time: 17.5 ms


In [49]:
%%time
# **** Get Expirations and Strikes. (3 mins)
contracts = [(i.symbol, '', i.secType, i.conId) for i in qual_stock]
contracts_p = [ib.reqSecDefOptParams(*p) for p in contracts]

# Filter symbol, expiration and strike from chains and make a cartesian
chains = [(i.tradingClass, i.expirations, i.strikes) for elem in contracts_p for i in elem]
L = [[[x[0]], sorted(x[1]), sorted(x[2])] for x in chains]

stock_chains = pd.DataFrame([j for i in L for j in product(*i)], 
                            columns=['Symbol', "Expiry", "Strike"])

df = pd.merge(stock_chains, nse_options, how='left', on='Symbol')

Wall time: 3min 59s


In [ ]:
%%time
# **** Get the top 2 out-of-SD-band contracts for each symbol, expiry and right ****

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

# Populate dte for standard deviation
df['DTE'] = [get_dte(d) for d in df.Expiry]
df['DTEMax'] = df.DTE.max()
df['Exchange'] = 'NSE'

# Make iSymbol unique for contracts
sd_df = df.drop_duplicates(subset=['iSymbol', 'DTE']).reset_index(drop=True)

# function to get greatest standard deviation between dte and dte_max
def get_stdev(contract, dte, dte_max):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
       (dte_max)  = int: max days to expiry
    Returns:
       maximum standard deviation in days (int)
    '''
    sd_days = str(dte_max)+' D'
    max_bars = ib.reqHistoricalData(contract=contract, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    
    sd_dte_max = np.std([b.close for b in max_bars], ddof=1)    # standard deviation of max dte
    sd_dte = np.std([b.close for b in max_bars[:-dte]], ddof=1)  # standard deviation of dte
    
    return max(sd_dte_max, sd_dte)  # returns the greatest standard deviation

sd_df['SD'] = pd.Series([get_stdev(c, d, x) for c, d, x in zip(sd_df.Contract, sd_df.DTE, sd_df.DTEMax)])

# Merge Standard Deviation to dataframe
df1 = pd.merge(df, sd_df, on=['iSymbol', 'Expiry'], how='left', suffixes=('', '_'))

df1 = df1.loc[:, ~df1.columns.str.endswith('_')].drop(['DTEMax'], axis=1)

# Designate Puts and Calls
df1['Right'] = np.where(df1.Strike < df1.Und_Price, 'P', 'C')

# Determine the Band Price
df1['Band_Price'] = np.where(df1.Right == 'P', df1.Und_Price - (df1.SD * put_sd), df1.Und_Price + (df1.SD * call_sd))

# Weed out-of-Band_Price strikes
mask = ((df1.Right == 'P') & (df1.Strike < df1.Band_Price)) | \
((df1.Right == 'C') & (df1.Strike > df1.Band_Price))

df2 = df1.loc[mask, :].reset_index(drop=True)  # Out-of-Band_Price

# Keep two out-of-band contracts for each right, expiry and symbol

df2["Band-Strike"] = pd.Series(abs(df2.Band_Price-df2.Strike)) # Calculate the absolute difference

In [132]:
df2 = df2.sort_values(by=['Symbol', 'Expiry', 'Right', 'Band-Strike']) # Sort
df3 = df2.groupby(['Symbol', 'Expiry', 'Right']).head(2).reset_index(drop=True) # Choose the top 2

# df3 = df3.head()          # !Data-Limiter

### Get the Margins and Prices

# Option contract list
c = [ib.qualifyContracts(Option(symbol=s, exchange=x, lastTradeDateOrContractMonth=d, strike=t, right=r)) for s, x, d, t, r in 
 zip(df3.iSymbol,df3.Exchange, df3.Expiry,df3.Strike,df3.Right)]

# Orders
o = [Order(action='SELL', orderType='MKT', totalQuantity=q) for q in df3.Mlot]

# In the DataFrame
df3['Options'] = pd.Series(c)
df3['Orders'] = pd.Series(o)

# Margins
df3['Margin'] = [pd.to_numeric(ib.whatIfOrder(*c, o).maintMarginChange) for c, o in zip(c,o)]

# Option Price
df3['OptPrice'] = pd.Series([get_price(*i) for i in df3.Options])

# Return on Margin
df3['RoM'] = df3.OptPrice*df3.Mlot/df3.Margin*365/df3.DTE

ERROR:ib_insync.wrapper:Error 200, reqId 22529: No security definition has been found for the request, contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20180927', strike=690.0, right='C', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20180927', strike=690.0, right='C', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 22530: No security definition has been found for the request, contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20180927', strike=570.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20180927', strike=570.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 22533: No security definition has been found for the request, contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20181025', strike=690.0, right='C', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Optio

ERROR:ib_insync.ib:Unknown contract: Option(symbol='GLENMARK', lastTradeDateOrContractMonth='20180927', strike=670.0, right='C', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23151: No security definition has been found for the request, contract: Option(symbol='GLENMARK', lastTradeDateOrContractMonth='20180927', strike=530.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='GLENMARK', lastTradeDateOrContractMonth='20180927', strike=530.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23152: No security definition has been found for the request, contract: Option(symbol='GLENMARK', lastTradeDateOrContractMonth='20181025', strike=670.0, right='C', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='GLENMARK', lastTradeDateOrContractMonth='20181025', strike=670.0, right='C', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23155: No security definition has been found for the request, contract: Option(sym

ERROR:ib_insync.wrapper:Error 200, reqId 23524: No security definition has been found for the request, contract: Option(symbol='INDUSINDB', lastTradeDateOrContractMonth='20181025', strike=1880.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='INDUSINDB', lastTradeDateOrContractMonth='20181025', strike=1880.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23578: No security definition has been found for the request, contract: Option(symbol='IRB', lastTradeDateOrContractMonth='20180927', strike=265.0, right='C', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='IRB', lastTradeDateOrContractMonth='20180927', strike=265.0, right='C', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23579: No security definition has been found for the request, contract: Option(symbol='IRB', lastTradeDateOrContractMonth='20180927', strike=155.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='IRB', 

ERROR:ib_insync.ib:Unknown contract: Option(symbol='MFSL', lastTradeDateOrContractMonth='20180927', strike=450.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23869: No security definition has been found for the request, contract: Option(symbol='MFSL', lastTradeDateOrContractMonth='20181025', strike=590.0, right='C', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='MFSL', lastTradeDateOrContractMonth='20181025', strike=590.0, right='C', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 23872: No security definition has been found for the request, contract: Option(symbol='MFSL', lastTradeDateOrContractMonth='20181025', strike=450.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='MFSL', lastTradeDateOrContractMonth='20181025', strike=450.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 24220: No security definition has been found for the request, contract: Option(symbol='RAYMOND', lastT

ERROR:ib_insync.ib:Unknown contract: Option(symbol='TORNTPHAR', lastTradeDateOrContractMonth='20180927', strike=1480.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 24617: No security definition has been found for the request, contract: Option(symbol='TORNTPHAR', lastTradeDateOrContractMonth='20181025', strike=1820.0, right='C', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='TORNTPHAR', lastTradeDateOrContractMonth='20181025', strike=1820.0, right='C', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 24619: No security definition has been found for the request, contract: Option(symbol='TORNTPHAR', lastTradeDateOrContractMonth='20181025', strike=1480.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='TORNTPHAR', lastTradeDateOrContractMonth='20181025', strike=1480.0, right='P', exchange='NSE')


ValueError: Unable to parse string "1.7976931348623157E308" at position 0

In [133]:
df3

,Symbol,Expiry,Strike,Mlot,iSymbol,Desc,Exchange,Contract,Und_Price,DTE,OptPrice,SD,Right,Band_Price,Band-Strike,Options,Orders
0,ACC,20180830,1760.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,C,1749.281082,10.718918,"[Option(conId=320081053, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
1,ACC,20180830,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,C,1749.281082,30.718918,"[Option(conId=325076677, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
2,ACC,20180830,1320.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,P,1320.532527,0.532527,"[Option(conId=320080883, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
3,ACC,20180830,1300.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,P,1320.532527,20.532527,"[Option(conId=320080872, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
4,ACC,20180927,1760.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,C,1749.281082,10.718918,"[Option(conId=324994936, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
5,ACC,20180927,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,C,1749.281082,30.718918,"[Option(conId=325076687, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
6,ACC,20180927,1320.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,P,1320.532527,0.532527,"[Option(conId=324346735, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
7,ACC,20180927,1300.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,P,1320.532527,20.532527,"[Option(conId=324346725, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
8,ACC,20181025,1760.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,78,NaN,103.811843,C,1749.281082,10.718918,"[Option(conId=327703267, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."
9,ACC,20181025,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,78,NaN,103.811843,C,1749.281082,30.718918,"[Option(conId=327703273, symbol='ACC', lastTra...","Order(action='SELL', totalQuantity=400, orderT..."


In [131]:
df3 = df3.sort_values(by=['RoM'], ascending=False)
calls_df = df3.loc[df3.Right == 'C', :]
puts_df = df3.loc[df3.Right == 'P', :]

x  ==  datetimedatetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_NSERaw.xlsx"
writer = pd.ExcelWriter(x)
df3.to_excel(writer, 'options', index=False, freeze_panes=(1, 1))
calls_df.to_excel(writer, 'calls', index=False, freeze_panes=(1,1))
puts_df.to_excel(writer, 'puts', index=False, freeze_panes=(1,1))
writer.save()

,Symbol,Expiry,Strike,Mlot,iSymbol,Desc,Exchange,Contract,Und_Price,DTE,OptPrice,SD,Right,Band_Price,Band-Strike
0,ACC,20180830,1760.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,C,1749.281082,10.718918
1,ACC,20180830,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,C,1749.281082,30.718918
2,ACC,20180830,1320.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,P,1320.532527,0.532527
3,ACC,20180830,1300.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,22,NaN,103.811843,P,1320.532527,20.532527
4,ACC,20180927,1760.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,C,1749.281082,10.718918
5,ACC,20180927,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,C,1749.281082,30.718918
6,ACC,20180927,1320.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,P,1320.532527,0.532527
7,ACC,20180927,1300.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,50,NaN,103.811843,P,1320.532527,20.532527
8,ACC,20181025,1760.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,78,NaN,103.811843,C,1749.281082,10.718918
9,ACC,20181025,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1524.00,78,NaN,103.811843,C,1749.281082,30.718918
